In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors



In [2]:
### Storage Setting
Ts = 1/12
c = 20
eta = 0.9

In [3]:
### Load Training Performance
base_path = 'models/BAW0EDH1MC20.0/Region{}/models_profit_julia_100_lr001.csv'

# Define your regions
Regions = [1,2,3,4,5,6]

# Initialize a dictionary to hold your DataFrames
ESmodelTrain = {}

# Loop over each region and read the corresponding CSV file
for region in Regions:
    # Format the file path with the current region
    file_path = base_path.format(region)
    
    # Read the CSV file and save it into the dictionary
    ESmodelTrain[region-1] = pd.read_csv(file_path)

In [13]:
### Bid Ahead
# case = 'Strategic/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio1.0_MIP0.1_DARTDP_Hete'
# case = 'Strategic/BidAhead/UC25ED1_Strategic_true_Seg1_Load1.0_Fuel1.2_Error0.25_ratio1.0_MIP0.1_DARTDP_BAW36'
# case = 'Strategic/MarginalCost/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC10.0'
case = 'Strategic/EDH/UC25ED13_Strategic_false_ratio1.0_Seg1_BAW0_MC20.0_1'
# case = 'Strategic/2022/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC20.0_1'

ESD = pd.read_csv('output/'+case+'/EDESD.csv',header=None)
ESC = pd.read_csv('output/'+case+'/EDESC.csv',header=None)
price = pd.read_csv('output/'+case+'/EDprice.csv',header=None)
ES = pd.read_csv('2032 ADS PCM V2.4.1 Public Data/Processed Data/7Regions/Storage_C_4hr_5GW_Strategic.csv')
# ES = pd.read_csv('2032 ADS PCM V2.4.1 Public Data/Processed Data/2022/Storage_C_4hr_5GW_Strategic.csv')
# ESmodel = pd.read_csv('output/'+case+'/Strategic/storage_to_index_map.csv')

In [14]:
HeteroData = []
for i in ESmodel['StorageID']:
    index = i-1
    Zone = ES['Zone'][index]
    Capacity = ES['MaxCap(MWh)'][index]
    PowerCap = ES['MaxCap(MW)'][index]
    Revenue = sum((ESD[index]-ESC[index])*price[ES['Zone'][index]-1])/1000
    Profit = sum((ESD[index]-ESC[index])*price[ES['Zone'][index]-1] - Ts*c*ESD[index])/1000
    UnitProfit = Profit/Capacity
#     ModelID = ESmodel[ESmodel['StorageID']==i]['SelectedModelIndex'].item()
#     TrainProfit = ESmodelTrain[ES['Zone'][index]-1]['Training profit'][ModelID-1]/1000
    
    HeteroData.append({
        'Storage ID': i,
        'Zone': Zone,
        'Capacity': Capacity,
        'PowerCapacity': PowerCap,
        'Revenue': Revenue,
        'Profit': Profit,
        'UnitProfit': UnitProfit,
#         'ModelID': ModelID,
#         'TrainProfit': TrainProfit
    })
HeteroDatadf = pd.DataFrame(HeteroData)
Zone1ES = HeteroDatadf[HeteroDatadf['Zone']==1]
Zone2ES = HeteroDatadf[HeteroDatadf['Zone']==2]
Zone3ES = HeteroDatadf[HeteroDatadf['Zone']==3]

In [15]:
# Initialize an empty dictionary to store results
UnitProfit = {}

# Calculate weighted average of UnitProfit based on Capacity for each Zone (1 through 6)
for zone in range(1, 7):
    zone_df = HeteroDatadf[HeteroDatadf['Zone'] == zone]
    if not zone_df.empty:  # Check if the DataFrame is not empty
        weighted_unit_profit = sum(zone_df['UnitProfit'] * zone_df['Capacity']) / sum(zone_df['Capacity'])
    else:
        weighted_unit_profit = None  # In case there's no data for the zone
    UnitProfit[f'Zone{zone}ES'] = weighted_unit_profit

# Convert the results dictionary to a single-row DataFrame
UnitProfit_df = pd.DataFrame([UnitProfit])

UnitProfit_df

,Zone1ES,Zone2ES,Zone3ES,Zone4ES,Zone5ES,Zone6ES
0,4.472621,3.740304,4.326931,6.111607,5.217282,5.965479


In [ ]:
### Temporary exlcude Zone 4
HeteroDatadf = HeteroDatadf[HeteroDatadf['Zone']!=4]

# Define a mapping of Zone to markers
zone_markers = {
    1.0: 'o',  # Circle
    2.0: 's',  # Square
    3.0: '^',  # Triangle up
    4.0: '>',  # Triangle right
    5.0: '<',  # Triangle left
    6.0: 'p',  # Pentagon
}

# Assuming HeteroDatadf is already defined

plt.figure(figsize=(10, 6))

# Normalize 'Capacity' for the entire dataset to get a consistent color mapping
norm = mcolors.Normalize(vmin=HeteroDatadf['PowerCapacity'].min(), vmax=HeteroDatadf['PowerCapacity'].max())

# Get the colormap
cmap = plt.get_cmap('viridis')

# Plot points for each zone with respective markers, color normalized by 'Capacity'
for zone in sorted(HeteroDatadf['Zone'].unique()):
    subset = HeteroDatadf[HeteroDatadf['Zone'] == zone]
    colors = cmap(norm(subset['PowerCapacity']))
    plt.scatter(subset['TrainProfit'], subset['UnitProfit'], marker=zone_markers[zone], c=colors, alpha=0.5)

# Adding a color bar that reflects the full range of 'Capacity' values across all zones
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, alpha=0.5)
cbar.set_label('Capacity (MW)')

# Annotations remain the same
for i in range(len(HeteroDatadf)):
    plt.text(HeteroDatadf['TrainProfit'].iloc[i], HeteroDatadf['UnitProfit'].iloc[i], f" {HeteroDatadf['ModelID'].iloc[i]}", fontsize=9, verticalalignment='bottom')

# Adjust legend for zone markers
legend_entries = [plt.Line2D([0], [0], color='black', marker=zone_markers[zone], linestyle='None', markersize=10, label=f'Zone {int(zone)}') for zone in sorted(HeteroDatadf['Zone'].unique())]
plt.legend(handles=legend_entries, title="Zones")

plt.title('Train Profit vs. Unit Profit (100% AI ES) with Capacity Colors and Zone Markers')
plt.xlabel('Train Profit ($/kWh)')
plt.ylabel('Unit Profit ($/kWh)')
plt.grid(True)
plt.ylim([0, 17])

plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
# Using capacity as the color variable
scatter = plt.scatter(HeteroDatadf['TrainProfit'], HeteroDatadf['UnitProfit'], marker='x', c=HeteroDatadf['PowerCapacity'], cmap='viridis', alpha=0.5)

# Adding a color bar to indicate the capacity values
cbar = plt.colorbar(scatter)
cbar.set_label('Capacity (MW)')

# Annotating each point with its capacity number (without "MWh" unit in the annotation)
for i in range(len(HeteroDatadf)):
    plt.text(HeteroDatadf['TrainProfit'].iloc[i], HeteroDatadf['UnitProfit'].iloc[i], f" {HeteroDatadf['ModelID'].iloc[i]}", fontsize=9)

# Adding a legend to explain the annotations
plt.legend([scatter], ['ModelID'], loc="upper right", title="Annotations")

plt.title('Train Profit vs. Unit Profit (100% AI ES) with Capacity Colors and Zone Markers')

plt.xlabel('Train Profit ($/kWh)')
plt.ylabel('Unit Profit ($/kWh)')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
# Using capacity as the color variable
scatter = plt.scatter(Zone2ES['TrainProfit'], Zone2ES['UnitProfit'], marker='x', c=Zone2ES['PowerCapacity'], cmap='viridis', alpha=0.5)

# Adding a color bar to indicate the capacity values
cbar = plt.colorbar(scatter)
cbar.set_label('Capacity (MW)')

# Annotating each point with its capacity number (without "MWh" unit in the annotation)
for i in range(len(Zone2ES)):
    plt.text(Zone2ES['TrainProfit'].iloc[i], Zone2ES['UnitProfit'].iloc[i], f" {Zone2ES['ModelID'].iloc[i]}", fontsize=9)

# Adding a legend to explain the annotations
plt.legend([scatter], ['ModelID'], loc="upper right", title="Annotations")

plt.title('Train Profit vs. Unit Profit (100% AI ES) with Capacity Colors')

plt.xlabel('Train Profit ($/kWh)')
plt.ylabel('Unit Profit ($/kWh)')
plt.grid(True)
plt.show()

### Benchmark 7 Regions Profit

In [ ]:
### Heterogeneous Storage Profit
ESD = pd.read_csv('output/Strategic/MC20/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC20.0/EDESD.csv',header=None)
ESC = pd.read_csv('output/Strategic/MC20/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC20.0/EDESC.csv',header=None)
price = pd.read_csv('output/Strategic/MC20/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC20.0/EDprice.csv',header=None)
ES = pd.read_csv('2032 ADS PCM V2.4.1 Public Data/Processed Data/7Regions/Storage_C_4hr_5GW.csv')
ESmodel = pd.read_csv('output/Strategic/MC20/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC20.0/Strategic/storage_to_index_map.csv')
# ESmodelTrain = pd.read_csv('models/models_profit_julia_2_100_lr001.csv')

In [ ]:
HeteroData = []
for i in ESmodel['StorageID']:
    index = i-1
    Zone = ES['Zone'][index]
    Capacity = ES['MaxCap(MWh)'][index]
    PowerCap = ES['MaxCap(MW)'][index]
    Revenue = sum((ESD[index]-ESC[index])*price[ES['Zone'][index]-1])/1000
    Profit = sum((ESD[index]-ESC[index])*price[ES['Zone'][index]-1] - Ts*c*ESD[index])/1000
    UnitProfit = Profit/Capacity
    ModelID = ESmodel[ESmodel['StorageID']==i]['SelectedModelIndex'].item()
    TrainProfit = ESmodelTrain[ES['Zone'][index]-1]['Training profit'][ModelID-1]/1000
    
    HeteroData.append({
        'Storage ID': i,
        'Zone': Zone,
        'Capacity': Capacity,
        'PowerCapacity': PowerCap,
        'Revenue': Revenue,
        'Profit': Profit,
        'UnitProfit': UnitProfit,
        'ModelID': ModelID,
        'TrainProfit': TrainProfit
    })
HeteroDatadf = pd.DataFrame(HeteroData)
Zone1ES = HeteroDatadf[HeteroDatadf['Zone']==1]
Zone2ES = HeteroDatadf[HeteroDatadf['Zone']==2]
Zone3ES = HeteroDatadf[HeteroDatadf['Zone']==3]

In [ ]:
# Initialize an empty dictionary to store results
UnitProfit = {}

# Calculate weighted average of UnitProfit based on Capacity for each Zone (1 through 6)
for zone in range(1, 7):
    zone_df = HeteroDatadf[HeteroDatadf['Zone'] == zone]
    if not zone_df.empty:  # Check if the DataFrame is not empty
        weighted_unit_profit = sum(zone_df['UnitProfit'] * zone_df['Capacity']) / sum(zone_df['Capacity'])
    else:
        weighted_unit_profit = None  # In case there's no data for the zone
    UnitProfit[f'Zone{zone}ES'] = weighted_unit_profit

# Convert the results dictionary to a single-row DataFrame
UnitProfit_df = pd.DataFrame([UnitProfit])

UnitProfit_df

In [ ]:
### Temporary exlcude Zone 4
HeteroDatadf = HeteroDatadf[HeteroDatadf['Zone']!=4]

# Define a mapping of Zone to markers
zone_markers = {
    1.0: 'o',  # Circle
    2.0: 's',  # Square
    3.0: '^',  # Triangle up
    4.0: '>',  # Triangle right
    5.0: '<',  # Triangle left
    6.0: 'p',  # Pentagon
}

# Assuming HeteroDatadf is already defined

plt.figure(figsize=(10, 6))

# Normalize 'Capacity' for the entire dataset to get a consistent color mapping
norm = mcolors.Normalize(vmin=HeteroDatadf['PowerCapacity'].min(), vmax=HeteroDatadf['PowerCapacity'].max())

# Get the colormap
cmap = plt.get_cmap('viridis')

# Plot points for each zone with respective markers, color normalized by 'Capacity'
for zone in sorted(HeteroDatadf['Zone'].unique()):
    subset = HeteroDatadf[HeteroDatadf['Zone'] == zone]
    colors = cmap(norm(subset['PowerCapacity']))
    plt.scatter(subset['TrainProfit'], subset['UnitProfit'], marker=zone_markers[zone], c=colors, alpha=0.5)

# Adding a color bar that reflects the full range of 'Capacity' values across all zones
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, alpha=0.5)
cbar.set_label('Capacity (MW)')

# Annotations remain the same
for i in range(len(HeteroDatadf)):
    plt.text(HeteroDatadf['TrainProfit'].iloc[i], HeteroDatadf['UnitProfit'].iloc[i], f" {HeteroDatadf['ModelID'].iloc[i]}", fontsize=9, verticalalignment='bottom')

# Adjust legend for zone markers
legend_entries = [plt.Line2D([0], [0], color='black', marker=zone_markers[zone], linestyle='None', markersize=10, label=f'Zone {int(zone)}') for zone in sorted(HeteroDatadf['Zone'].unique())]
plt.legend(handles=legend_entries, title="Zones")

plt.title('Train Profit vs. Unit Profit (100% AI ES) with Capacity Colors and Zone Markers')
plt.xlabel('Train Profit ($/kWh)')
plt.ylabel('Unit Profit ($/kWh)')
plt.ylim([0, 17])
plt.grid(True)

plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
# Using capacity as the color variable
scatter = plt.scatter(Zone2ES['TrainProfit'], Zone2ES['UnitProfit'], marker='x', c=Zone2ES['PowerCapacity'], cmap='viridis', alpha=0.5)

# Adding a color bar to indicate the capacity values
cbar = plt.colorbar(scatter)
cbar.set_label('Capacity (MWh)')

# Annotating each point with its capacity number (without "MWh" unit in the annotation)
for i in range(len(Zone2ES)):
    plt.text(Zone2ES['TrainProfit'].iloc[i], Zone2ES['UnitProfit'].iloc[i], f" {Zone2ES['ModelID'].iloc[i]}", fontsize=9)

# Adding a legend to explain the annotations
plt.legend([scatter], ['ModelID'], loc="upper right", title="Annotations")

plt.title('Train Profit vs. Unit Profit (100% AI ES) with Capacity Colors')

plt.xlabel('Train Profit ($/kWh)')
plt.ylabel('Unit Profit ($/kWh)')
plt.grid(True)
plt.show()

In [ ]:
### Price Analysis
# price = pd.read_csv('output/Strategic/Scale/UC25ED1_Strategic_true_Seg5_Load1.0_Fuel1.2_Error0.25_ratio0.5_MIP0.1_DARTDP/EDprice.csv',header=None)
# df = pd.concat([price.mean()*12, (price*12).std()], axis=1)
# df.T

In [ ]:
### Emission
ThermalGen = pd.read_csv('2032 ADS PCM V2.4.1 Public Data/Processed Data/ThermalGen_Full_C.csv')
case = 'Strategic/MC20/UC25ED1_Strategic_true_ratio1.0_Seg1_BAW0_MC20.0'
EDGen = pd.read_csv('output/'+case+'/EDGen.csv',header=None)

In [ ]:
def distribute_output_across_segments(interval_output, thermalgen_row):
    """
    Distribute a single interval's output across the generator's operational segments.

    :param interval_output: The total output for the generator in a 5-minute interval.
    :param thermalgen_row: A pandas Series containing the generator's segment data.
    :return: A list where each element represents the output allocated to a segment.
    """
    # Initialize variables
    segment_outputs = []  # Stores the output allocated to each segment
    remaining_output = interval_output
    
    # Base segment output
    if remaining_output > 0:
        base_output = min(remaining_output, thermalgen_row['IOMinCap(MW)'])
        segment_outputs.append(base_output)
        remaining_output -= base_output
    else:
        segment_outputs.append(0)

    # Incremental segments
    for i in range(2, 7):  # Assuming up to 6 segments
        inc_cap_key = f'IncCap{i}(MW)'
        if inc_cap_key in thermalgen_row.index and thermalgen_row[inc_cap_key] > 0:
            if remaining_output > 0:
                segment_capacity = thermalgen_row[inc_cap_key]
                segment_output = min(remaining_output, segment_capacity)
                segment_outputs.append(segment_output)
                remaining_output -= segment_output
            else:
                segment_outputs.append(0)
        else:
            break  # No more segments defined

    # If there are less than 6 segments, fill the remaining with 0s
    while len(segment_outputs) < 6:
        segment_outputs.append(0)

    return segment_outputs

In [ ]:
# Initialize a dictionary to store the interval outputs for each segment
interval_segment_outputs = {gen_id: [] for gen_id in EDGen.columns}

for gen_id in EDGen.columns:
    thermalgen_row = ThermalGen.loc[gen_id]  # Adjust this line based on how you match generators
    
    for interval_output in EDGen[gen_id]:
        segment_outputs = distribute_output_across_segments(interval_output, thermalgen_row)
        interval_segment_outputs[gen_id].append(segment_outputs)

In [ ]:
# Placeholder for the aggregated hourly segment outputs
hourly_segment_outputs = {gen_id: pd.DataFrame(columns=[f'Segment{i}' for i in range(1, 7)]) for gen_id in EDGen.columns}

for gen_id, outputs in interval_segment_outputs.items():
    # Convert list of outputs to DataFrame for easier manipulation
    outputs_df = pd.DataFrame(outputs, columns=[f'Segment{i}' for i in range(1, 7)])
    
    # Aggregate to hourly by summing every 12 rows (12 intervals per hour)
    hourly_data = outputs_df.groupby(outputs_df.index // 12).sum()
    
    hourly_segment_outputs[gen_id] = hourly_data

In [ ]:
def map_emission_factor(fuel_name):
    if fuel_name.startswith('Coal'):
        return 95.92
    elif fuel_name.startswith('NG'):
        return 52.91
    elif fuel_name.startswith('Oil'):
        return 74.14
    elif fuel_name == 'Petroleum Coke':
        return 102.41
    elif fuel_name == 'Geothermal' or fuel_name == 'Uranium':
        return 0
    elif fuel_name == 'Waste_Heat':
        return 66.33
    elif fuel_name in ['Bio_Landfill_Gas', 'Bio_Other']:
        return 52.07
    elif fuel_name == 'Bio_Wood':
        return 93.80
    elif fuel_name == 'Bio_Blk_Liquor':
        return 73.84
    elif fuel_name == 'Bio_Solid_Waste':
        return 90.70
    elif fuel_name == 'Bio_Agri_Res':
        return 118.17
    elif fuel_name == 'DefaultFuel':
        return 52.91
    else:
        return None  # Or a default value if you have one for unknown types

# Apply the function to create a new column
ThermalGen['Emission Factor'] = ThermalGen['Fuel Name'].apply(map_emission_factor)

# Check if there's any fuel type that did not get an emission factor (None values)
missing_emission_factors = ThermalGen[ThermalGen['Emission Factor'].isnull()]
if not missing_emission_factors.empty:
    print("Missing emission factors for:", missing_emission_factors['Fuel Name'].unique())
else:
    print("All fuel types have assigned emission factors.")

In [ ]:
# Initialize a DataFrame to store hourly total emissions for all generators
hourly_total_emissions = pd.DataFrame(index=hourly_segment_outputs[0].index)

for gen_id in hourly_segment_outputs.keys():
    thermalgen_row = ThermalGen.loc[gen_id]
    hourly_data = hourly_segment_outputs[gen_id]
    emission_factor = thermalgen_row['Emission Factor']
    
    # Initialize a Series to store total hourly fuel consumption for this generator
    hourly_fuel_consumption = pd.Series(0, index=hourly_data.index)

    # Calculate fuel consumption for each segment
    for i in range(1, 7):  # Assuming up to 6 segments
        segment_key = f'Segment{i}'
        if i == 1:
            # Base segment calculation
            base_fuel = (hourly_data[segment_key] * thermalgen_row['MinInput(MMBTu)'] / thermalgen_row['IOMinCap(MW)']).fillna(0) / 12
            hourly_fuel_consumption += base_fuel
        else:
            # Incremental segments
            hr_key = f'IncHR{i}(MMBTu/MWh)'
            if hr_key in thermalgen_row.index and thermalgen_row[hr_key] > 0:
                incremental_fuel = (hourly_data[segment_key] * thermalgen_row[hr_key]).fillna(0) / 12
                hourly_fuel_consumption += incremental_fuel

    # Calculate total hourly emissions for this generator
    hourly_emissions = hourly_fuel_consumption * emission_factor
    hourly_total_emissions[gen_id] = hourly_emissions

hourly_total_emissions.head()

In [ ]:
# Sum across rows and plot
row_sums = hourly_total_emissions.sum(axis=1)/1000
window_size = 24  # For example, for hourly data, a 24-hour (daily) moving average
smoothed_sums = row_sums.rolling(window=window_size, center=True).mean()

plt.figure(figsize=(12, 7))
plt.plot(row_sums.index, row_sums.values, label='Original', alpha=0.5)  # Making the original line semi-transparent
plt.plot(smoothed_sums.index, smoothed_sums.values, label='Smoothed (Moving Average)', linewidth=2)

plt.title('Hourly Total Emissions with Moving Average Smoothing (start up not added)')
plt.xlabel('Hour')  # Adjust label if your index has a different meaning
plt.ylabel('Total Emissions (ton)')
plt.legend()
plt.grid(True)
plt.show()